In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [12]:
matchups = pd.read_csv("data\\matchups\\model_matchups2.csv", index_col=0)

In [13]:
y = matchups['outcome']
x_train = matchups.drop(columns=['outcome','game_code','batter_game_code','pitcher_game_code','batter_id','pitcher_id','b_pos'])

In [14]:
y.unique()

array(['GO', '2B', '1B', 'BB', 'HBP', 'SO', 'HR', 'E', 'IBB', '3B', 'DG'],
      dtype=object)

In [15]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
labels_enc = le.transform(y)
labels = tf.keras.utils.to_categorical(labels_enc)

ss = preprocessing.StandardScaler()
x_standardized = ss.fit_transform(x_train)

In [32]:
p = .1
inputs = tf.keras.layers.Input((74,), name='numeric_inputs')

x = tf.keras.layers.Dropout(p)(inputs)
x = tf.keras.layers.Dense(1500, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(250, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(50, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
out = tf.keras.layers.Dense(11, activation='softmax', name='output')(x)

In [33]:
model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 numeric_inputs (InputLayer)  [(None, 74)]             0         
                                                                 
 dropout_28 (Dropout)        (None, 74)                0         
                                                                 
 dense_23 (Dense)            (None, 1500)              112500    
                                                                 
 batch_normalization_23 (Bat  (None, 1500)             6000      
 chNormalization)                                                
                                                                 
 dropout_29 (Dropout)        (None, 1500)              0         
                                                                 
 dense_24 (Dense)            (None, 500)               750500    
                                                           

In [34]:
def bootstrap_sample_generator(batch_size):
    while True:
        batch_idx = np.random.choice(x_standardized.shape[0], batch_size)
        yield ({'numeric_inputs': x_standardized[batch_idx]},
               {'output': labels[batch_idx]})

batch_size = 128

model.fit(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=10_000 // batch_size,
    epochs=10,
    max_queue_size=10  
)


Epoch 1/10
78/78 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.1429
Epoch 2/10
78/78 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.1473
Epoch 3/10
78/78 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.1464
Epoch 4/10
78/78 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.1459
Epoch 5/10
78/78 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.1457
Epoch 6/10
78/78 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.1421
Epoch 7/10
78/78 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.1367
Epoch 8/10
78/78 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.1407
Epoch 9/10
78/78 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.1377
Epoch 10/10
78/78 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.1339
